In [17]:
import pandas as pd
!pip install fuzzywuzzy
!pip install python-Levenshtein

In [18]:
from fuzzywuzzy import fuzz
value = fuzz.token_sort_ratio('New York in the house 2002 uhgbjkn hubijnk', 'hgbjn in the house New York! ')
print('value: ' + str(value))

value: 64


In [32]:
Data = pd.read_csv("C:/Users/hrida/Documents/Coding/PythonProjects/Market-Analysis-Apriori/combined.csv")
Data

,No,Bill number,Date,Item,Code,HSN Code,Unit,Qty
0,13617/21-22,1361722,01/01/22,Cadbury - Oreo Chocolatey Sandwich Biscuits - ...,3095,19053100.0,PAC,2.0
1,13618/21-22,1361822,01/01/22,Surf Excel Washing Soap Bar - 80 gms,1898,34011930.0,PAC,1.0
2,13619/21-22,1361922,01/01/22,Vim - Lemons - 145 ml,2491,34022010.0,PAC,2.0
3,13619/21-22,1361922,01/01/22,Nescafe - Classic Coffee Sachet - 7.5 gms,3043,210111.0,PAC,3.0
4,13620/21-22,1362022,01/01/22,B Fizz- 160 ml,2435,NaN,CAN,1.0
...,...,...,...,...,...,...,...,...
265582,40148/22-23,4014823,30/09/22,Catch - Fruit Juice - Apple - 200 ml Bottle,5517,22029020.0,BTL,2.0
265583,40149/22-23,4014923,30/09/22,Catch-Flavoured drink- Orange,5835,22021020.0,PCS,1.0
265584,40150/22-23,4015023,30/09/22,Epigamia - Milkshake - Strawberry - 180 ml,4696,22029930.0,PAC,1.0
265585,40150/22-23,4015023,30/09/22,Galaxy Crispy - 18 gms,5878,18069010.0,PAC,1.0


In [21]:
unique_items = Data['Item'].unique()
print(len(unique_items))

similar=[]
for i in unique_items:
    if fuzz.token_sort_ratio(i,"Real Cranberry Juice - 1 Ltr")>60:
        similar.append(i)

similar

6307


['REal Fruit Juice - Orange - 180 Lit.',
 'Real Fruit Juice - Orange - 01 Lit.',
 'Real Fruit Juice - Guava - 180 ml',
 'Real Cranberry Juice - 1 Ltr',
 'Real Juice - Guava - 200 ml',
 'Real Juice - Cranberry - 180 ml',
 'Real Apple Juice - 180 ml',
 'Dabur - Aloe Vera Juice - 01 Lit.',
 'Real Mango Juice - 1 Ltr',
 'Real 100% Active Apple Juice - 1 Ltr',
 'Real Fruit Power - Cranberry - 1 Lit.',
 'Real Guava Juice - 1 Ltr',
 'Real Juice - Guava - 180 ml',
 'Real - Fruit Juice - Mosambi - 1 Lit.',
 'Real - Apple Juice - 125 ml']

## APPLY FUZZY STRING MATCHING

In [23]:
excl_merged = pd.read_csv("C:/Users/hrida/Documents/Coding/PythonProjects/Market-Analysis-Apriori/combined.csv")

In [24]:
excl_merged['Bill number'].unique()

array([1361722, 1361822, 1361922, ..., 4014823, 4014923, 4015023],
      dtype=int64)

In [25]:
# change product Code data type

excl_merged = excl_merged.astype({"Code": int})

In [27]:
nan_in_col  = excl_merged[excl_merged['Code'].isna()]
nan_in_col

,No,Bill number,Date,Item,Code,HSN Code,Unit,Qty


In [28]:
excl_merged.dtypes

No              object
Bill number      int64
Date            object
Item            object
Code             int32
HSN Code       float64
Unit            object
Qty            float64
dtype: object

In [33]:
item_master = {}
for i in range(excl_merged.shape[0]):
	item_master[excl_merged['Code'][i]]=excl_merged['Item'][i]

print(item_master)

{3095: 'Cadbury - Oreo Chocolatey Sandwich Biscuits - Original - 50 gms', 1898: 'Surf Excel Washing Soap Bar - 80 gms', 2491: 'Vim - Lemons - 145 ml', 3043: 'Nescafe - Classic Coffee Sachet - 7.5 gms', 2435: 'B Fizz- 160 ml', 1800: 'Parle - Monaco Classic Biscuits - 75.4 gms', 350: 'Haldiram Chips Pudina Treat 65.5 gm', 600: 'Dukes Waffy - Chocolate - 75 gms', 198: 'Uttam - Sulphur Free Sugar - 1 kg', 1771: 'Orbit Spearmint Chewing Gum - 4.4 gms', 1535: 'REal Fruit Juice - Orange - 180 Lit.', 411: 'Real Mixed Fruit Juice - 180 ml', 547: 'Real Fruit Power - Pineapple - 180 ml', 780: 'Indiagate Dubar Rice - 5Kg', 1086: 'Fortune Soyabean Refined Oil - 1 Lit.', 624: 'Pringles - Originals - 107 gms', 421: 'Parle Milano Choco Delight -75 gms', 351: 'Haldiram Bhelpuri 200gm', 2913: "Haldiram's - Chips - Tangy Tomato - 65.5 gms", 1642: 'Veto Plast - Plastic Buckets - 16 Lit.', 805: 'Aucto  Plastic Mug - 1.5 Lit.', 1424: 'Parle -G Biscuits - 50+10=60 gms', 89: 'Banana ( Kela )', 1164: 'Dove Sha

In [41]:
from fuzzywuzzy import fuzz

similar_groups = {}
compared_codes = []
counter = 0
for code in item_master.keys():
    names = []
    final_code = -1
    if counter % 10 == 0:
        print("Processed Items: ", counter)
    for code_to_compare in item_master.keys():
        if code not in compared_codes:
            match_value = fuzz.token_sort_ratio(item_master[code], item_master[code_to_compare])
            if match_value > 90:
                names.append(item_master[code_to_compare])
                final_code = code
    counter += 1
    compared_codes.append(code)
    similar_groups[final_code] = names
    
similar_groups

Processed Items:  0
Processed Items:  10
Processed Items:  20
Processed Items:  30
Processed Items:  40


In [ ]:
reduced_similar_groups = []
for i in range(len(similar_groups)):
	if len(similar_groups[i])!=1:
		reduced_similar_groups.append(similar_groups[i])

In [72]:

for i in reduced_similar_groups:
	print(str(i) + "\n\n")

['Surf Excel Washing Soap Bar - 80 gms', 'Surf Excel Washing Soap Bar - 95 gms']


['Vim - Lemons - 145 ml', 'Vim - Lemons - 155 ml']


['Haldiram Chips Pudina Treat 65.5 gm', 'Haldiram,s - Pudina Ttreat Chips - 60.5 gms']


['REal Fruit Juice - Orange - 180 Lit.', 'Real Fruit Juice - Orange - 01 Lit.']


['Real Mixed Fruit Juice - 180 ml', 'Real Mixed Fruit Juice - 125 ml']


['Pringles - Originals - 107 gms', 'Pringles Original - 107 gms']


['Aucto  Plastic Mug - 1.5 Lit.', 'Aucto VM - Plastic Mug - 1.5 Lit.']


['Dove Shampoo - Hair Fall Rescue - 180 ml', 'Dove - Shampoo - Hair Fall Rescue - 180 ml', 'Dove Shampoo - Hair Fall Rescue  - 80 ml']


['Maggi Masala Noodles - 280 gms', 'Maggi - Masala Noodles - 140 gms', 'Maggi - Masala Noodles - 280 gms', 'Maggi Masala Noodles - 420gms', 'Maggi - Masala Noodles - 280 gms', 'Maggi - Masala Noodles - 280 gms', 'Maggi - Masala Noodles - 32 gms', 'Maggi Noodles - Masala - 420 gms']


['Coca Cola - 250 ml', 'Coca Cola -750 ml']


['Veeba Egg